### Хакатон AI.Hack СПб. Churn Detection
### Газпром Нефть

https://www.kaggle.com/c/ai-hackathon-2018-spb-gpn

Количество колонок трейна и теста не отличается, пробуем посмотрть на тесте, если связь churn с тем, что нет последнего месяца по клиенту в датасете

In [10]:
import pandas as pd
import numpy as np

In [11]:
test = pd.read_csv('datasets/gpn/test_data.csv', low_memory=False)
sample_submission = pd.read_csv('datasets/gpn/sample_submission.csv', low_memory=False)

In [12]:
test.sample(3)

,Unnamed: 0,time,date,v_l,q,n_tr,sum_b,code_azs,id,first_prch,location,region,code,code1,percent,type
4113794,4113794,NaN,2016-12-30,18.18,0,2016,661.82,1574.0,21-8AH-17929,28.01.16 11:11:45,8.0,455.0,01010131,128.0,0.00,5
1751686,1751686,19:15:54,2016-03-10,22.51,0,2016,713.68,14118.0,21-8AE-2226,19.01.16 13:20:04,7.0,456.0,01010121,126.0,21.41,7
1786617,1786617,NaN,2017-02-16,17.34,0,2017,745.46,13987.0,21-8B2-6395,11.12.16 21:17:33,7.0,455.5,1010142,133.0,0.00,13


In [13]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31122 entries, 0 to 31121
Data columns (total 2 columns):
id       31122 non-null object
proba    31122 non-null float64
dtypes: float64(1), object(1)
memory usage: 486.4+ KB


In [14]:
test = test[test.v_l > 0] # оставляем только тех, кто платил за топливо
test.shape

(3701008, 16)

In [15]:
df = test[['date','id']]

In [16]:
res = df.groupby('id').max()

В декабре было совсем пусто, попробуем считать как churn не имеющих транзакций в ноябре

In [17]:
res['proba'] = res['date'].apply(lambda x: 1 if str(x) <= '2017-10-31' else 0)

In [18]:
res.shape, sample_submission.shape

((31109, 2), (31122, 2))

In [19]:
def id_to_proba(id):
    try:
        return res[res.index == str(id)]['proba'][0]
    except:
        return 1

In [20]:
submission = sample_submission
submission.sample(3)

,id,proba
3437,21-8AC-11911,0.5
4998,21-8AD-4376,0.5
16893,21-19DS-85,0.5


In [21]:
%%time
submission['proba'] = submission['id'].apply(id_to_proba)

CPU times: user 2min 18s, sys: 551 ms, total: 2min 19s
Wall time: 2min 19s


In [22]:
submission.sample(3)

,id,proba
821,21-1886-1971,0
24904,21-8AH-17483,1
27452,21-8AN-6187,1


In [23]:
submission.to_csv('gpn-sub.csv', index=False)